In [1]:
# install packages
%pip install boto3
%pip install sagemaker
%pip install -U sagemaker
%pip install torchmetrics

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [44]:
# cell 01
import os
import sagemaker
import torchmetrics
import torch
from sagemaker import get_execution_role


sagemaker_session = sagemaker.Session()

role = 'AmazonSageMaker-ExecutionRole-20220714T204241'
# role = get_execution_role()
region = sagemaker_session.boto_session.region_name

In [45]:
# cell 02
training_data_uri = 's3://multibert-8b87f872-a3fe-4a19-a016-e02402275450/training/multi_label_new.csv'
#test_data_uri = 's3://multibert-8b87f872-a3fe-4a19-a016-e02402275450/test'

In [46]:
from sagemaker.pytorch import PyTorch

In [47]:
# cell 05
pytorch_estimator = PyTorch('multibert.py',
                            instance_type='ml.c5.2xlarge',
                            instance_count=2,
                            role = role,
                            framework_version='1.11.0',
                            py_version='py38',
                            source_dir = 'code',
                            hyperparameters = {'epochs': 4, 'batch-size': 32, 'learning-rate': 2e-05})


In [48]:
# cell 07
pytorch_estimator.fit(training_data_uri)
#pytorch_estimator.fit({'train': training_data_uri,
#                       'test': test_data_uri})

2022-07-17 00:20:39 Starting - Starting the training job...
2022-07-17 00:20:54 Starting - Preparing the instances for trainingProfilerReport-1658017238: InProgress
......
2022-07-17 00:22:13 Downloading - Downloading input data
2022-07-17 00:22:13 Training - Downloading the training image........bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-07-17 00:23:30,774 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-07-17 00:23:30,776 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-07-17 00:23:30,782 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-07-17 00:23:30,787 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-07-17 00:23:30,793 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-07-17 00:23:31,106 sagemaker-training-tool

UnexpectedStatusException: Error for Training job pytorch-training-2022-07-17-00-20-36-989: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
ExitCode 1
ErrorMessage "NameError: name 'BERT_MODEL_NAME' is not defined
"
Command "/opt/conda/bin/python3.8 multibert.py --batch-size 32 --epochs 4 --learning-rate 2e-05", exit code: 1

In [ ]:
# cell 09
predictor = pytorch_estimator.deploy(instance_type='ml.m4.xlarge',
                                     initial_instance_count=1)

In [ ]:
# cell 11
data = "I am a test."
response = predictor.predict(data)